# Przetwarzanie wstępne. Filtracja kontekstowa.


### Cel:
- zapoznanie z pojęciem kontekstu / filtracji kontekstowej,
- zapoznanie z pojęciem konwolucji (splotu),
- zapoznanie z wybranymi filtrami:
	- filtry liniowe dolnoprzepustowe:
		- filtr uśredniający,
		- filtr Gaussa.
	- filtry nielinowe:
		- mediana,
		- mediana dla obrazów kolorowych.
	- filtry liniowe górnoprzepustowe:
			- laplasjan,
			- operator Robersta, Prewitta, Sobela.
- zadanie domowe: adaptacyjna filtracja medianowa.

### Filtry liniowe uśredniające (dolnoprzepustowe)

Jest to podstawowa rodzina filtrów stosowana w cyfrowym przetwarzaniu obrazów. 
Wykorzystuje się je w celu "rozmazania" obrazu i tym samym redukcji szumów (zakłóceń) na obrazie.
Filtr określony jest przez dwa parametry: rozmiar maski (ang. _kernel_) oraz wartości współczynników maski.

Warto zwrócić uwagę, że omawiane w niniejszym rozdziale operacje generują nową wartość piksela na podstawie pewnego fragmentu obrazu (tj. kontekstu), a nie jak operacje punktowe tylko na podstawie jednego piksela.


1. Wczytaj obraz _plansza.png_.
W dalszej części ćwiczenia sprawdzenie działania filtracji dla innych obrazów sprowadzi się do wczytania innego pliku.

2. Podstawowa funkcja to `cv2.filter2D`  - realizacja filtracji konwolucyjnej.
   Proszę sprawdzić jej dokumentację i zwrócić uwagę na obsługę problemu brzegowego (na krawędziach istnieją piksele dla których nie da się wyznaczyć otoczenia).

  Uwaga. Problem ten można też rozwiązać z użyciem funkcji `signal.convolve2d` z biblioteki _scipy_ (`from scipy import signal`).

3. Stwórz podstawowy filtr uśredniający o rozmiarze $3 \times 3$ -- za pomocą funkcji `np.ones`. Wykonaj konwolucję na wczytanym obrazie. Na wspólnym rysunku wyświetl obraz oryginalny, po filtracji oraz moduł z różnicy.

4. Przeanalizuj otrzymane wyniki. Jakie elementy zawiera obraz "moduł z różnicy"? Co na tej podstawie można powiedzieć o filtracji dolnoprzepustowej?

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from scipy import signal


# Obrazki
if not os.path.exists("jet.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/jet.png --no-check-certificate
if not os.path.exists("kw.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/kw.png --no-check-certificate
if not os.path.exists("moon.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/moon.png --no-check-certificate
if not os.path.exists("lenaSzum.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lenaSzum.png --no-check-certificate
if not os.path.exists("lena.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lena.png --no-check-certificate
if not os.path.exists("plansza.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/plansza.png --no-check-certificate


jet = cv2.imread('jet.png', cv2.IMREAD_GRAYSCALE)
kw = cv2.imread('kw.png', cv2.IMREAD_GRAYSCALE)
moon = cv2.imread('moon.png', cv2.IMREAD_GRAYSCALE)
lenasz = cv2.imread('lenaSzum.png', cv2.IMREAD_GRAYSCALE)
lena = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)
plansza = cv2.imread('plansza.png', cv2.IMREAD_GRAYSCALE)

In [ ]:
kernel = np.ones(shape = (3, 3)) * (1/9)
plansza_filter2D = cv2.filter2D(plansza, -1, kernel)
plansza_convolved2d = signal.convolve2d(plansza, kernel, mode='same')

def disp_img_and_filtered(img, filtered):
    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(24, 8)
    ax[0].imshow(img, 'gray', vmin=0, vmax=256)
    ax[0].axis("off")
    ax[0].set_title("Obraz oryginalny")

    ax[1].imshow(filtered, 'gray', vmin=0, vmax=256)
    ax[1].axis("off")
    ax[1].set_title("Obraz przefiltrowany")
    
    mod = np.abs(img - filtered)
    ax[2].imshow(mod, 'gray', vmin=0, vmax=256)
    ax[2].axis("off")
    ax[2].set_title("Moduł z różnicy")

disp_img_and_filtered(plansza, plansza_filter2D)
disp_img_and_filtered(plansza, plansza_convolved2d)

Analizując przefiltrowany obraz cieżko jest gołym okiem zauważyć różnicę pomiędzy działaniem funkcji filter2D, a convolve2d (poza obecnością bardzo wąskiej ramki dla drugiej funkcji). Inaczej się ma sprawa z modułem z różnicy - tutaj dla pierwszej funkcji widzimy wyraźne kontury każdego elementu, natomiast dla convolve2d są one zależne od koloru elementu (mogą być bardziej lub mniej wyraźne). Warto też zwrócić uwagę na różnicę w szumach na module z różnicy - w pierwszym obrazie szum jest rozmyty, nayomiast w drugim - dosyć wyraźny.

5. Na wspólnym rysunku wyświetl wyniki filtracji uśredniającej z oknem o rozmiarze 3, 5, 9, 15 i 35. 
Wykorzystaj polecenie `plt.subplot`. 
Przeanalizuj wpływ rozmiaru maski na wynik. 

In [ ]:
def disp_different_sizes(img):
    fig, ax = plt.subplots(1, 6)
    fig.set_size_inches(24, 4)
    i = 1
    ax[0].imshow(img, 'gray', vmin=0, vmax=256)
    ax[0].axis("off")
    ax[0].set_title("Obraz oryginalny")
    for size in [3, 5, 9, 15, 35]:
        kernel = np.ones(shape = (size, size)) / (size**2)
        img_filt = cv2.filter2D(img, -1, kernel)
        ax[i].imshow(img_filt, 'gray', vmin=0, vmax=256)
        ax[i].axis("off")
        ax[i].set_title("Rozmiar maski = " + str(size))
        i += 1

disp_different_sizes(plansza)

Zwiększanie rozmiaru maski powoduje zwiększenie rozmycia obrazu.

6. Wczytaj obraz _lena.png_.
Zaobserwuj efekty filtracji dolnoprzepustowej dla obrazu rzeczywistego.

In [ ]:
disp_different_sizes(lena)

Zgodnie z oczekiwaniami obraz przefiltrowany staje się bardziej rozmyty. Dla rozmiaru większego niż 5 rozmycie zaburza widoczność detali.

7. Niekorzystny efekt towarzyszący wykonanym filtracjom dolnoprzepustowym to utrata ostrości. 
Częściowo można go zniwelować poprzez odpowiedni dobór maski. 
Wykorzystaj maskę:  `M = np.array([1 2 1; 2 4 2; 1 2 1])`. 
Przed obliczeniami należy jeszcze wykonać normalizację - podzielić każdy element maski przez sumę wszystkich elementów: `M = M/sum(sum(M));`.
Tak przygotowaną maskę wykorzystaj w konwolucji - wyświetl wyniki tak jak wcześniej.
Możliwe jest też wykorzystywanie innych masek - współczynniki można dopasowywać do konkretnego problemu.

In [ ]:
M = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
M = M/sum(sum(M))

lena_convolved2d = signal.convolve2d(lena, M, mode='same')

def disp_img_and_filt(img, filtered):
    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(16, 8)
    ax[0].imshow(img, 'gray', vmin=0, vmax=256)
    ax[0].axis("off")
    ax[0].set_title("Obraz oryginalny")

    ax[1].imshow(filtered, 'gray', vmin=0, vmax=256)
    ax[1].axis("off")
    ax[1].set_title("Obraz przefiltrowany")

disp_img_and_filt(lena, lena_convolved2d)

Obraz został wygładzony bez dużej strat ostrości.

8. Skuteczną i często wykorzystywaną maską jest tzw. maska Gasussa.
Jest to zbiór liczb, które aproksymują dwuwymiarowy rozkład Gaussa. 
Parametrem jest odchylenie standardowe i rozmiar maski.

9. Wykorzystując przygotowaną funkcję `fgaussian` stwórz maskę o rozmiarze $5 \times 5$ i odchyleniu standardowym 0.5.
  Wykorzystując funkcję `mesh` zwizualizuj filtr.
  Sprawdź jak parametr ``odchylenie standardowe'' wpływa na ``kształt'' filtru.

  Uwaga. W OpenCV dostępna jest *dedykowana* funkcja do filtracji Gaussa - `GaussianBlur`.
  Proszę na jednym przykładzie porównać jej działanie z użytym wyżej rozwiązaniem.

10. Wykonaj filtrację dla wybranych (2--3) wartości odchylenia standardowego.


In [ ]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g/g.sum() 

def mesh(fun, size):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    ax.plot_surface(X, Y, Z)
    plt.show()
    
mask1 = fgaussian(5, 0.1)
mask2 = fgaussian(5, 0.5)
mask3 = fgaussian(5, 3)
mesh(mask1, 5)
mesh(mask2, 5)
mesh(mask3, 5)

lena_1 = signal.convolve2d(lena, mask1, mode='same')
lena_2 = signal.convolve2d(lena, mask2, mode='same')
lena_3 = signal.convolve2d(lena, mask3, mode='same')
lena_gauss_cv = cv2.GaussianBlur(lena, (5,5), 3)
disp_img_and_filt(lena, lena_1)
disp_img_and_filt(lena, lena_2)
disp_img_and_filt(lena, lena_3)
disp_img_and_filt(lena, lena_gauss_cv)

Zwiększanie odchylenia standardowego Zwiększa rozmycie obrazu, a wizualizacja filtru jest mniej 'ostra', wypukłość jest bardziej rozłożona na wszystkie strony, a nie skoncentrowana w środku. Dodatkowo, im większe odchylenie, tym mniejsze są wartości na osi z. Zaimplementowana tutaj funkcja wydaje się dawać takie same eekty, jak wbudowana funkcjia GaussianBlur z pakietu cv2.

### Filtry nieliniowe -- mediana

Filtry rozmywające redukują szum, ale niekorzystnie wpływają na ostrość obrazu.
Dlatego często wykorzystuje się filtry nieliniowe - np. filtr medianowy (dla przypomnienia: mediana - środkowa wartość w posortowanym ciągu liczb).

Podstawowa różnica pomiędzy filtrami liniowymi, a nieliniowymi polega na tym, że przy filtracji liniowej na nową wartość piksela ma wpływ wartość wszystkich pikseli z otoczenia (np. uśrednianie, czasem ważone), natomiast w przypadku filtracji nieliniowej jako nowy piksel wybierana jest któraś z wartości otoczenia - według jakiegoś wskaźnika (wartość największa, najmniejsza czy właśnie mediana).


1. Wczytaj obraz _lenaSzum.png_ (losowe 10% pikseli białych lub czarnych - tzw. zakłócenia impulsowe). Przeprowadź filtrację uśredniającą z rozmiarem maski 3x3. Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Wykorzystując funkcję ``cv2.medianBlur` wykonaj filtrację medianową _lenaSzum.png_ (z rozmiarem maski $3 \times 3$). Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Która filtracja lepiej radzi sobie z tego typu szumem?

  Uwaga. Taki sam efekt da również użycie funkcji `signal.medfilt2d`.


In [ ]:
lenasz_convolved2d = signal.convolve2d(lenasz, kernel, mode='same')
lenasz_medianblurr = cv2.medianBlur(lenasz, 3)
disp_img_and_filtered(lenasz, lenasz_convolved2d)
disp_img_and_filtered(lenasz, lenasz_medianblurr)

Filtracja medianowa dała znacznie lepszy rezultat. Moduł z różnicy zawiera także kontury obrazu, a nie tylko sam szum. Pierwszy sposób filtracji złagodził szum (rozmył go), jednak nie został on usunięty.

2. Przeprowadź filtrację uśredniającą, a następnie medianową obrazu _lena.png_.
   Wyniki porównaj - dla obu wyświetl: oryginał, wynik filtracji i moduł z różnicy.
   Szczególną uwagę zwróć na ostrość i krawędzie.
   W której filtracji krawędzie zostają lepiej zachowane?

In [ ]:
lena_convolved2d = signal.convolve2d(lena, kernel, mode='same')
lena_medianblurr = cv2.medianBlur(lena, 3)
disp_img_and_filtered(lena, lena_convolved2d)
disp_img_and_filtered(lena, lena_medianblurr)

Lepiej zachowane krawędzie i ostrość występują dla drugiej funkcji - korzystającej z mediany.

3. Ciekawy efekt można uzyskać wykonując filtrację medianową wielokrotnie. Określa się go mianem  posteryzacji.  W wyniku przetwarzania z obrazka usunięte zostają detale, a duże obszary uzyskują tą samą wartość jasności.  Wykonaj operację mediany $5 \times 5$ na obrazie _lena.png_ 10-krotnie. (wykorzystaj np. pętlę `for`).


Inne filtry nieliniowe:
- filtr modowy - moda (dominanta) zamiast mediany,
- filtr olimpijski - średnia z podzbioru otoczenia (bez wartości ekstremalnych),
- hybrydowy filtr medianowy - mediana obliczana osobno w różnych podzbiorach otoczenia (np. kształt ``x'',``+''), a jako wynik brana jest mediana ze zbioru wartość elementu centralnego, mediana z ``x'' i mediana z ``+'',
- filtr minimalny i maksymalny (będą omówione przy okazji operacji morfologicznych w dalszej części kursu).


Warto zdawać sobie sprawę, z szerokich możliwości dopasowywania rodzaju filtracji do konkretnego rozważanego problemu i rodzaju zaszumienia występującego na obrazie.

In [ ]:
lena_10 = cv2.medianBlur(lena, 5)
for i in range(10):
    lena_10 = cv2.medianBlur(lena_10, 5)

disp_img_and_filtered(lena, lena_10)

Obraz został rozmyty w dosyć charakterystyczny sposób, gdyż istotniejsze krawędzie wciąż są wyraźne.

## Filtry liniowe górnoprzepustowe (wyostrzające, wykrywające krawędzie)

Zadaniem filtrów górnoprzepustowych jest wydobywanie z obrazu składników odpowiedzialnych za szybkie zmiany jasności - konturów, krawędzi, drobnych elementów tekstury.

### Laplasjan (wykorzystanie drugiej pochodnej obrazu)

1. Wczytaj obraz _moon.png_.

2. Wprowadź podstawową maskę laplasjanu:
\begin{equation}
M = 
\begin{bmatrix}
0 & 1& 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0
\end{bmatrix}
\end{equation}

3. Przed rozpoczęciem obliczeń należy dokonać normalizacji maski - dla rozmiaru $3 \times 3$ podzielić każdy element przez 9.
   Proszę zwrócić uwagę, że nie można tu zastosować takiej samej normalizacji, jak dla filtrów dolnoprzepustowanych, gdyż skutkowałby to dzieleniem przez 0.

4. Wykonaj konwolucję obrazu z maską (`c2.filter2D`). Przed wyświetleniem, wynikowy obraz należy poddać normalizacji (występują ujemne wartości). Najczęściej wykonuje się jedną z dwóch operacji:
- skalowanie (np. poprzez dodatnie 128 do każdego z pikseli),
- moduł (wartość bezwzględna).

Wykonaj obie normalizacje. 
Na wspólnym wykresie wyświetl obraz oryginalny oraz przefiltrowany po obu normalizacjach. 

In [ ]:
M_l = np.array([[0, 1/9, 0], [1/9, -4/9, 1/9], [0, 1/9, 0]], dtype=np.float32)
moon_1 = np.ones(shape = moon.shape, dtype=np.float32)
cv2.filter2D(moon.astype(np.float32), -1, M_l, dst = moon_1)
moon_2 = moon_1 + 128
moon_3 = np.abs(moon_1)

def disp_img_and_filt_no0256(img, filtered):
    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(16, 8)
    ax[0].imshow(img, 'gray', vmin=0, vmax=256)
    ax[0].axis("off")
    ax[0].set_title("Obraz oryginalny")

    ax[1].imshow(filtered, 'gray')
    ax[1].axis("off")
    ax[1].set_title("Obraz przefiltrowany")

disp_img_and_filt_no0256(moon, moon_2)
disp_img_and_filt_no0256(moon, moon_3)

7. Efekt wyostrzenia uzyskuje się po odjęciu/dodaniu (zależy do maski) rezultatu filtracji laplasjanowej i oryginalnego obrazu. Wyświetl na jednym wykresie: obraz oryginalny, sumę oryginału i wyniku filtracji oraz różnicę (bezwzględną) oryginału i wyniku filtracji.
 Uwaga. Aby uniknąć artefaktów, należy obraz wejściowy przekonwertować do formatu ze znakiem.



In [ ]:
def disp_img_and_sum_and_mod(img, filtered):
    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(24, 8)
    ax[0].imshow(img, 'gray', vmin=0, vmax=256)
    ax[0].axis("off")
    ax[0].set_title("Obraz oryginalny")

    ax[1].imshow(img+filtered, 'gray')
    ax[1].axis("off")
    ax[1].set_title("Suma obrazu i wyniku filtracji")
    
    mod = np.abs(img - filtered)
    ax[2].imshow(mod, 'gray', vmin=0, vmax=256)
    ax[2].axis("off")
    ax[2].set_title("Moduł z różnicy")

disp_img_and_sum_and_mod(moon, moon_2)
disp_img_and_sum_and_mod(moon, moon_3)

### Gradienty (wykorzystanie pierwszej pochodnej obrazu)

1. Wczytaj obraz _kw.png_. Stwórz odpowiednie maski opisane w kolejnych punktach i dokonaj filtracji.
2. Wykorzystując gradient Robertsa przeprowadź detekcję krawędzi - poprzez wykonanie konwolucji obrazu z daną maską:
\begin{equation}
R1 = \begin{bmatrix} 0 & 0 & 0 \\ -1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix}   
R2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & -1 \\ 0 & 1 & 0 \end{bmatrix}
\end{equation}

Wykorzystaj stworzony wcześniej kod (przy laplasjanie) - dwie metody normalizacji oraz sposób wyświetlania.

3. Analogicznie przeprowadź detekcję krawędzi za pomocą gradientu Prewitta (pionowy i poziomy)
\begin{equation}
P1 = \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}   
P2 = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}
\end{equation}

4. Podobnie skonstruowany jest gradient Sobela (występuje osiem masek, zaprezentowane są dwie ``prostopadłe''):
\begin{equation}
S1 = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}   
S2 = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
\end{equation}

Przeprowadź detekcję krawędzi za pomocą gradientu Sobela. 

In [ ]:
R1 = np.array([[0, 0, 0], [-1, 0, 0], [0, 1, 0]])
R2 = np.array([[0, 0, 0], [0, 0, -1], [0, 1, 0]])
P1 = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
P2 = P1.T
S1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
S2 = S1.T

kw_1 = np.ones(shape = kw.shape, dtype=np.float32)
cv2.filter2D(kw.astype(np.float32), -1, R1, dst = kw_1)
kw_2 = kw_1 + 128
kw_3 = np.abs(kw_1)

kw_4 = np.ones(shape = kw.shape, dtype=np.float32)
cv2.filter2D(kw.astype(np.float32), -1, R2, dst = kw_4)
kw_5 = kw_4 + 128
kw_6 = np.abs(kw_4)

kw_7 = np.ones(shape = kw.shape, dtype=np.float32)
cv2.filter2D(kw.astype(np.float32), -1, P1, dst = kw_7)
kw_8 = kw_7 + 128
kw_9 = np.abs(kw_7)

kw_10 = np.ones(shape = kw.shape, dtype=np.float32)
cv2.filter2D(kw.astype(np.float32), -1, P2, dst = kw_10)
kw_11 = kw_10 + 128
kw_12 = np.abs(kw_10)

kw_13 = np.ones(shape = kw.shape, dtype=np.float32)
cv2.filter2D(kw.astype(np.float32), -1, S1, dst = kw_13)
kw_14 = kw_13 + 128
kw_15 = np.abs(kw_13)

kw_16 = np.ones(shape = kw.shape, dtype=np.float32)
cv2.filter2D(kw.astype(np.float32), -1, S2, dst = kw_16)
kw_17 = kw_16 + 128
kw_18 = np.abs(kw_16)

disp_img_and_filt_no0256(kw, kw_2)
disp_img_and_filt_no0256(kw, kw_3)
disp_img_and_filt_no0256(kw, kw_5)
disp_img_and_filt_no0256(kw, kw_6)
disp_img_and_filt_no0256(kw, kw_8)
disp_img_and_filt_no0256(kw, kw_9)
disp_img_and_filt_no0256(kw, kw_11)
disp_img_and_filt_no0256(kw, kw_12)
disp_img_and_filt_no0256(kw, kw_14)
disp_img_and_filt_no0256(kw, kw_15)
disp_img_and_filt_no0256(kw, kw_17)
disp_img_and_filt_no0256(kw, kw_18)

disp_img_and_sum_and_mod(kw, kw_2)
disp_img_and_sum_and_mod(kw, kw_3)
disp_img_and_sum_and_mod(kw, kw_5)
disp_img_and_sum_and_mod(kw, kw_6)
disp_img_and_sum_and_mod(kw, kw_8)
disp_img_and_sum_and_mod(kw, kw_9)
disp_img_and_sum_and_mod(kw, kw_11)
disp_img_and_sum_and_mod(kw, kw_12)
disp_img_and_sum_and_mod(kw, kw_14)
disp_img_and_sum_and_mod(kw, kw_15)
disp_img_and_sum_and_mod(kw, kw_17)
disp_img_and_sum_and_mod(kw, kw_18)

Różne maski dają różne efekty dotyczące lini pionowych, poziomych, bądź ukośnych.

5. Na podstawie dwóch ortogonalnych masek np. Sobela można stworzyć tzw. filtr kombinowany - pierwiastek kwadratowy z sumy kwadratów gradientów:
\begin{equation}
OW = \sqrt{(O * S1)^2 + (O * S2)^2}
\end{equation}
gdzie:  $OW$ - obraz wyjściowy, $O$ - obraz oryginalny (wejściowy), $S1,S2$ - maski Sobela, $*$ - operacja konwolucji.

Zaimplementuj filtr kombinowany.

Uwaga. Proszę zwrócić uwagę na konieczność zmiany formatu danych obrazu wejściowego - na typ znakiem



In [ ]:
def comb(img):
    global S1
    global S2
    conv1 = cv2.filter2D(img.astype(np.float32), -1, S1)
    conv2 = cv2.filter2D(img.astype(np.float32), -1, S2)
    out = np.sqrt(conv1**2 + conv2**2)
    disp_img_and_sum_and_mod(img, out)

comb(kw)

6. Istnieje alternatywna wersja filtra kombinowanego, która zamiast pierwiastka z sumy kwadratów wykorzystuje sumę modułów (prostsze obliczenia). 
Zaimplementuj tę wersję. 

In [ ]:
def comb2(img):
    global S1
    global S2
    conv1 = cv2.filter2D(img.astype(np.float32), -1, S1)
    conv2 = cv2.filter2D(img.astype(np.float32), -1, S2)
    out = np.abs(conv1) + np.abs(conv2)
    disp_img_and_sum_and_mod(img, out)

comb2(kw)

Krawiędzie pionowe i poziome są dużo ostrzejsze, natomiast wokół ukosnych można zaobserwować dodatkową "ramkę".

7. Wczytaj plik _jet.png_ (zamiast _kw.png_).
Sprawdź działanie obu wariantów filtracji kombinowanej.

In [ ]:
comb(jet)
comb2(jet)

Krawędzie często zostały zastąpione kolorem białym, co w niektórych przypadkach zaburzyło ich widoczność.